In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split

data = pd.read_csv('../input/titanic/train.csv')
data = data.drop(['PassengerId' , 'Name' , 'Ticket' ,'Cabin' ] , axis = 'columns')
print(data.shape)
data.head()





In [ ]:

le =LabelEncoder()
label =le.fit_transform(data['Sex'])
data =data.drop(['Sex'] , axis = 'columns')
data['Sex'] = label

replacer =[]

for i in data['Embarked']:
    if i=='S':
        replacer.append(1)
    else:
        replacer.append(0)

data['Embarked'].replace('S',1)
data =data.drop(['Embarked'] , axis = 'columns')
data['Embarked'] = replacer

for col in data.columns:
    data[col] = data[col].fillna(data[col].mean())

data.head()

In [ ]:
def get_score(model , X_train , X_test , Y_train , Y_test):
    model.fit(X_train ,Y_train)
    return model.score(X_test ,Y_test)

In [ ]:
#classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier

modelNames=['DecisionTreeClassifier', 'RandomForestClassifier',
            'KNeighborsClassifier', 'GaussianNB', 
            'LogisticRegression', 'svm', 'MLPClassifier']
model = [DecisionTreeClassifier() , 
        RandomForestClassifier(),
       KNeighborsClassifier() ,  
       GaussianNB(),
       LogisticRegression(),
       svm.LinearSVC(),
        MLPClassifier(random_state=1, max_iter=300)
        ]
#evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
#feature selection 

from sklearn.feature_selection import SelectKBest, chi2 ,  RFE
X = data.drop(['Survived'] , axis = 'columns')
Y = data['Survived']

sel_f = SelectKBest(chi2, k = 7).fit(X,Y)
sel_bool =sel_f.get_support()
sel_f = X[X.columns[sel_bool]]
X =X[sel_f.columns]
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.20, random_state=0)

for i in model:
    if i == svm.LinearSVC():
        continue
    print(i, get_score(i,X_train, X_test, Y_train, Y_test ))


In [ ]:
# recursive fe

for i in model:
    if i==model[2]:
        continue
    if i==model[3]:
        continue
    if i==model[4]:
        continue
    
    
    selection  =  RFE(i , n_features_to_select = 3)
    selection.fit(X,Y)
    X_sel =  selection.transform(X)
    sel_bool=selection.get_support()
    sel_f = X[X.columns[sel_bool]]
    print(i, get_score(i,X_train, X_test, Y_train, Y_test ))



In [ ]:
# k fold cross validation 
from sklearn.model_selection import KFold , StratifiedKFold ,cross_val_score , LeaveOneOut , RepeatedKFold



kf = KFold(n_splits = 3) 
# for train_index , test_index, in kf.split([1,2,3,4,5,6,7,8,9]):
#     print(train_index ,test_index)
    

 

for train_index ,test_index in kf.split(X): 
    X_train, X_test ,Y_train   , Y_test =  X.loc[train_index] ,\
                   X.loc[test_index]  , Y.loc[train_index], Y.loc[test_index]
    
    for i in model:
        
        print(i , get_score(i  , X_train, X_test ,Y_train   , Y_test))


In [ ]:
# cross validation  score

for i in model:
    print(cross_val_score(i , X,Y ,scoring = 'accuracy' ,cv =  3))
    
#consluction  -the best model is randomforest classifier


 



In [ ]:
folds = StratifiedKFold(n_splits = 3)
scores=[]

for train_index, test_index in folds.split(X,Y):
    X_train, X_test ,Y_train   , Y_test =  X.loc[train_index] ,\
                   X.loc[test_index]  , Y.loc[train_index], Y.loc[test_index]
    
    for i in model:
        print(i , get_score(i  , X_train, X_test ,Y_train   , Y_test))
    

In [ ]:
start = time.time()
result = cross_val_score(estimator=model[1] , X=X,y=Y, scoring  ='accuracy' , cv = KFold(n_splits =5) )
print(result , time.time()-start)
# result = cross_val_score(estimator=model[0] , X=X,y=Y, scoring  ='accuracy' , cv = LeaveOneOut() )
start = time.time()
result = cross_val_score(estimator=model[1] , X=X,y=Y, scoring  ='accuracy' , cv = StratifiedKFold(n_splits =5) )
print(result , time.time()-start)
start = time.time()
result = cross_val_score(estimator=model[1] , X=X,y=Y, scoring  ='accuracy' , cv = RepeatedKFold(n_splits =5 , n_repeats =5) )
print(result , time.time()-start)